# Guitar Image Classification

### This notebook is part of the AiLab School AI cours. Contains the code to train a model to classify guitar images, part of the final project of the course.3

The dataset used in this project is derivated from the publication Fuzzy Edge-Detection as a Preprocessing Layer in Deep Neural Networks for Guitar Classification, contains images of 6 different guitar types.
<li>Acoustic</li>
<li>Double Cut</li>
<li>Single Cut</li>
<li>S-Style</li>
<li>T-Style</li>
<li>Ukulele</li>
The dataset was created from a web search and contains 900 images of each guitar type. The images were resized to 224x224 pixel.

In [1]:
# Tensorflow imports for CNN model
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from utils import ProcessingUtils



In [4]:
# Load the dataset
DATA_PATH = 'Y://Datasets//CGD//CGD_no_split//'
CLASSES = ['Acoustic', 'Double_cut', 'Single_cut', 'S_style', 'T_style', 'Ukulele']
X,y = ProcessingUtils.load_dataset(path=DATA_PATH, classes=CLASSES, img_size=(224, 224), shuffle=True, verbose=True)

Loading class Acoustic ...


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'Y://Datasets//CGD//CGD_no_split//Acoustic'